# 22: MTC Motorized and Non-Motorized Nested Mode Choice

In [ ]:
# TEST
import numpy as np
import pandas as pd

import larch as lx

pd.set_option("display.max_columns", 999)
pd.set_option("expand_frame_repr", False)
pd.set_option("display.precision", 3)

In [ ]:
import larch as lx

lx.__version__

In [ ]:
m = lx.example(17)
m.compute_engine = "numba"

For this example, we're going to re-create model 22 from the
[Self Instructing Manual](http://www.caee.utexas.edu/prof/Bhat/COURSES/LM_Draft_060131Final-060630.pdf).
(pp. 179)

In [ ]:
motorized = m.graph.new_node(
    parameter="mu_motor", children=[1, 2, 3, 4], name="Motorized"
)
nonmotorized = m.graph.new_node(
    parameter="mu_nonmotor", children=[5, 6], name="Nonmotorized"
)

In [ ]:
m.ordering = (
    (
        "CostbyInc",
        "costbyincome",
    ),
    (
        "TravelTime",
        ".*time.*",
        ".*dist.*",
    ),
    (
        "Household",
        "hhinc.*",
        "vehbywrk.*",
    ),
    (
        "Zonal",
        "wkcbd.*",
        "wkempden.*",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.graph

In [ ]:
mj = m.copy()
mj.compute_engine = "jax"

The numba and jax compute engines give us the same result, but they have
different performance characteristics: 

**numba**
- compile time is quite short and required only once per session,
- run time is pretty good
    
**jax**
- compile time is relatively long and required for every change in the model structure
- run time is blazing fast

For small models with relatively small data sets and simple structures,
the numba engine will typically be preferred.  As data size or model complexity
grows, the optimizations achievable in jax compilation become more valuable.

In [ ]:
result = m.maximize_loglike(method="bhhh")

In [ ]:
# TEST
r = result
from pytest import approx

assert r.loglike == approx(-3441.6725273276093, rel=1e-4)
assert dict(zip(m.pnames, r.x)) == approx(
    {
        "ASC_Bike": -1.2024073703132545,
        "ASC_SR2": -1.3257247399188727,
        "ASC_SR3+": -2.506936202874175,
        "ASC_Transit": -0.4041442821263078,
        "ASC_Walk": 0.3447191316975722,
        "costbyincome": -0.03864412094110262,
        "hhinc#4": -0.003929601755651791,
        "hhinc#5": -0.010035112486480783,
        "hhinc#6": -0.006205979434206748,
        "motorized_ovtbydist": -0.11389194935358538,
        "motorized_time": -0.014518697124195973,
        "mu_motor": 0.7261696323637531,
        "mu_nonmotor": 0.7690391629698575,
        "nonmotorized_time": -0.046200685225646534,
        "vehbywrk_Bike": -0.7347520730837045,
        "vehbywrk_SR": -0.22598417504565899,
        "vehbywrk_Transit": -0.7075038510739201,
        "vehbywrk_Walk": -0.7641289876632265,
        "wkcbd_Bike": 0.4077477599180845,
        "wkcbd_SR2": 0.1930608067123969,
        "wkcbd_SR3+": 0.7814124041724411,
        "wkcbd_Transit": 0.9217986579385763,
        "wkcbd_Walk": 0.11364443225208345,
        "wkempden_Bike": 0.0016747777566393732,
        "wkempden_SR2": 0.0011502120827767475,
        "wkempden_SR3+": 0.0016390812178071399,
        "wkempden_Transit": 0.0022379922179423173,
        "wkempden_Walk": 0.0021706844461508662,
    },
    rel=1e-2,
)

In [ ]:
m.calculate_parameter_covariance()
m.parameter_summary()

In [ ]:
# TEST
expected_se = pd.Series(
    {
        "ASC_Bike": 0.416852464751687,
        "ASC_SR2": 0.2545998857743335,
        "ASC_SR3+": 0.4749098839206808,
        "ASC_Transit": 0.2211891608590185,
        "ASC_Walk": 0.35780565829941885,
        "costbyincome": 0.010368875452431911,
        "hhinc#4": 0.0016122509691149048,
        "hhinc#5": 0.004650739659998643,
        "hhinc#6": 0.00302148217700312,
        "motorized_ovtbydist": 0.021102031065567364,
        "motorized_time": 0.003865662496250571,
        "mu_motor": 0.13491012665162105,
        "mu_nonmotor": 0.1785021270767945,
        "nonmotorized_time": 0.005396871957201883,
        "vehbywrk_Bike": 0.22879172995664934,
        "vehbywrk_SR": 0.06504869465180056,
        "vehbywrk_Transit": 0.14983034511610385,
        "vehbywrk_Walk": 0.1633867246456,
        "wkcbd_Bike": 0.3276503369966752,
        "wkcbd_SR2": 0.09619096122973834,
        "wkcbd_SR3+": 0.19983327839835419,
        "wkcbd_Transit": 0.2218432314826066,
        "wkcbd_Walk": 0.23643542277462148,
        "wkempden_Bike": 0.0010873335879477298,
        "wkempden_SR2": 0.00035425322602890654,
        "wkempden_SR3+": 0.0004487422174289541,
        "wkempden_Transit": 0.0005072868584578029,
        "wkempden_Walk": 0.0007623255600411431,
    },
    name="t_stat",
)
pd.testing.assert_series_equal(
    m.parameters.std_err.to_series(), expected_se, rtol=5.0e-2, check_names=False
)

In [ ]:
resultj = mj.maximize_loglike(stderr=True)

In [ ]:
# TEST
r = resultj
from pytest import approx

assert r.loglike == approx(-3441.6725273276093)
assert dict(zip(mj.pnames, r.x)) == approx(
    {
        "ASC_Bike": -1.2024073703132545,
        "ASC_SR2": -1.3257247399188727,
        "ASC_SR3+": -2.506936202874175,
        "ASC_Transit": -0.4041442821263078,
        "ASC_Walk": 0.3447191316975722,
        "costbyincome": -0.03864412094110262,
        "hhinc#4": -0.003929601755651791,
        "hhinc#5": -0.010035112486480783,
        "hhinc#6": -0.006205979434206748,
        "motorized_ovtbydist": -0.11389194935358538,
        "motorized_time": -0.014518697124195973,
        "mu_motor": 0.7261696323637531,
        "mu_nonmotor": 0.7690391629698575,
        "nonmotorized_time": -0.046200685225646534,
        "vehbywrk_Bike": -0.7347520730837045,
        "vehbywrk_SR": -0.22598417504565899,
        "vehbywrk_Transit": -0.7075038510739201,
        "vehbywrk_Walk": -0.7641289876632265,
        "wkcbd_Bike": 0.4077477599180845,
        "wkcbd_SR2": 0.1930608067123969,
        "wkcbd_SR3+": 0.7814124041724411,
        "wkcbd_Transit": 0.9217986579385763,
        "wkcbd_Walk": 0.11364443225208345,
        "wkempden_Bike": 0.0016747777566393732,
        "wkempden_SR2": 0.0011502120827767475,
        "wkempden_SR3+": 0.0016390812178071399,
        "wkempden_Transit": 0.0022379922179423173,
        "wkempden_Walk": 0.0021706844461508662,
    },
    rel=1e-2,
)
assert mj.pstderr == approx(
    np.array(
        [
            4.168558e-01,
            2.545745e-01,
            4.749014e-01,
            2.212087e-01,
            3.577894e-01,
            1.036857e-02,
            1.612386e-03,
            4.650531e-03,
            3.021172e-03,
            2.110203e-02,
            3.866330e-03,
            1.349138e-01,
            1.784958e-01,
            5.396660e-03,
            2.287765e-01,
            6.506080e-02,
            1.498349e-01,
            1.633791e-01,
            3.276509e-01,
            9.619242e-02,
            1.998420e-01,
            2.218623e-01,
            2.364309e-01,
            1.087425e-03,
            3.542897e-04,
            4.488141e-04,
            5.073145e-04,
            7.623227e-04,
        ],
        dtype=np.float32,
    ),
    rel=1e-2,
)